# notebook settings

In [2]:
# automatic code formating is available using the extension jupyterlab_code_formatter 
# the formatter is set to black (PEP 8 compliant)

# display full output when running a cell instead of only the last result
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "last"

# interactively display json
from IPython.display import JSON

# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Deliverables

- setup a datascience environment in the cloud
- X make code PEP8 compliant using black
- X Efficiently getting the job done
- X Choose meaningfull names for variables & functions
- X document
- X Help a data scientist present interactive results.
- X Offer predictions via REST api (cloud and local)
- X improve notebook layout

- notebook with the completed assignment
- list of packages for your runtime (e.g. yml or txt file)
- evidence of a working API endpoint

### Todo not-urgent
- install jupyter lab packages
- add setup to personal website





# Setup a datascience environment in the cloud

Information about the Jupyter Lab environment you are currently working in:  
    - hosted on AWS  
    - launched using Docker  
    - has jupyter lab extensions installed  
    

# Excercise - Data ingestion

## Imports

In [4]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import joblib

import requests
import json
from pprint import pprint
import unittest

## Getting store location data from an API

### Get data

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

### Explore the api

In [5]:
url = "https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places"
resp = requests.get(url)
d = resp.json()  # d is a list of dictionaries
df = json_normalize(data=d, sep=".")

In [6]:
# explore data
JSON(d[0])

<IPython.core.display.JSON object>

In [7]:
df.shape

(251, 21)

In [8]:
# set an index?
# df.placeId.nunique()
df.set_index("placeId", drop=True, inplace=True)
# optimize memory-usage by changing dtypes?
# df.dtypes
# not required, memory is not an issue

In [9]:
df.head(2)

,commercialName,branchId,sourceStatus,sellingPartners,handoverServices,moreInfoUrl,routeUrl,isActive,ensign.id,ensign.name,placeType.id,placeType.longName,placeType.placeTypeDescription,geoCoordinates.latitude,geoCoordinates.longitude,address.streetName,address.houseNumber,address.postalcode,address.cityName,address.countryName
placeId,,,,,,,,,,,,,,,,,,,,
902,AALST (COLRUYT),4156,AC,[QUALITY],[CSOP_ORDERABLE],https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,8,COLR_Colruyt,1,Winkel,Winkel,50.933074,4.053897,BRUSSELSE STEENWEG,41,9300,AALST,België
946,AALTER (COLRUYT),4218,AC,[QUALITY],[CSOP_ORDERABLE],https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,8,COLR_Colruyt,1,Winkel,Winkel,51.078476,3.450013,LOSTRAAT,66,9880,AALTER,België


### Create a function to get the data

In [10]:
def get_clp_places(url):
    resp = requests.get(url)
    d = resp.json()
    df = json_normalize(data=d, sep=".")
    df.set_index("placeId", drop=True, inplace=True)
    # optimize memory-usage by changing dtypes
    # memory is not an issue, changing dtypes is not yet required
    return df


url = "https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places"
df_clp = get_clp_places(url)
df_clp.head(2)

,commercialName,branchId,sourceStatus,sellingPartners,handoverServices,moreInfoUrl,routeUrl,isActive,ensign.id,ensign.name,placeType.id,placeType.longName,placeType.placeTypeDescription,geoCoordinates.latitude,geoCoordinates.longitude,address.streetName,address.houseNumber,address.postalcode,address.cityName,address.countryName
placeId,,,,,,,,,,,,,,,,,,,,
902,AALST (COLRUYT),4156,AC,[QUALITY],[CSOP_ORDERABLE],https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,8,COLR_Colruyt,1,Winkel,Winkel,50.933074,4.053897,BRUSSELSE STEENWEG,41,9300,AALST,België
946,AALTER (COLRUYT),4218,AC,[QUALITY],[CSOP_ORDERABLE],https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,8,COLR_Colruyt,1,Winkel,Winkel,51.078476,3.450013,LOSTRAAT,66,9880,AALTER,België


### Check data quality

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [11]:
records_min = 200
latitude_min = 49
latitude_max = 52
longitude_min = 4
# longitude_min = 2
longitude_max = 7
tc = unittest.TestCase("__init__")

# check records len
tc.assertTrue(len(df_clp.index) > records_min, f"less than {records_min} records")


def test_range(test_name, series, range_min=None, range_max=None):
    tc.assertTrue(
        series.min() > range_min, f"{test_name} ({series.min()}) < minimum({range_min})"
    )
    tc.assertTrue(
        series.max() < range_max, f"{test_name} ({series.max()}) > maximum({range_max})"
    )


# check latitude range
test_range(
    test_name="latitude",
    series=df_clp["geoCoordinates.latitude"],
    range_min=latitude_min,
    range_max=latitude_max,
)

# check longitude range
test_range(
    test_name="longitude",
    series=df_clp["geoCoordinates.longitude"],
    range_min=longitude_min,
    range_max=longitude_max,
)

# further improvement, return the placeId on a False

AssertionError: False is not true : longitude (2.6492578) < minimum(4)

### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [13]:
# Should a selection be done on 'placeType.longName'?
# df_clp['placeType.longName'].unique()
# No, the places are all "winkel"
# What is the province field?
# a province field does not exist!?
# Use the postcode to check for the province?
# on wikipedia https://www.wikiwand.com/en/List_of_postal_codes_in_Belgium -> Antwerp postcodes: 2000-2999

df_clp["address.postalcode"] = df_clp["address.postalcode"].astype("int32")
df_clp["antwerpen"] = np.where(
    (df_clp["address.postalcode"] >= 2000) & (df_clp["address.postalcode"] <= 2999),
    1,
    0,
)

# data quality check
df_clp["antwerpen"].value_counts()

0    216
1     35
Name: antwerpen, dtype: int64

# Excercise - REST-api for an ML-model

A datascientist made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.

In [1]:
def retrieve_model(path):
    trained_model = joblib.load(path)
    return trained_model


lgbr_cars = retrieve_model("rest_api\lgbr_cars.model")

# check the class of the model
tc = unittest.TestCase("__init__")
tc.assertEqual(
    str(type(lgbr_cars)), "<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars)
)

NameError: name 'joblib' is not defined

Functional test of the trained model.  
Parameters:

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804  
However, the model doesnt accept string inputs, see the integer encoding below:

In [16]:
model_test_input = [[3, 1, 190, -1, 125000, 5, 3, 1]]

In [17]:
def make_prediction(trained_model, single_input):
    predicted_value = trained_model.predict(single_input)[0]
    return predicted_value


predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)

### REST api

The lgbr_cars model REST api code is available in the car_price_prediction_app.py file.  

not in scope:
- authentication
- documentation
- nice inputs

AWS deployment 
The API is deployed as a Dockerized Flask web-app on AWS Elastic Beanstalk.  
The rest_api\Dockerfile is used in the EB deployment.
The 

REST api details:   
url: http://ml-demo-test1.us-east-1.elasticbeanstalk.com/predict
Should the url not work. Contact me to launch the server again.

Local deployment  
REST api details: 
- start a terminal
- activate the conda enviroment
- run "python car_price_prediction_app.py"
- the REST api is up and running
- test the api using the test instructions below
- shutdown the api "CTRL+c" in the terminal

The REST api can be tested using the following input and output:  
[-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

You can use Postman, curl or Python to test the API.  
[Postman](https://www.getpostman.com/)

**Curl:**
```bash
curl -d '{"single_input": [-1, 1, 0, 118, 150000, 0, 1, 38]}' -H "Content-Type: application/json" -X POST <url>
```

**Python:** 

In [ ]:
url = "http://127.0.0.2:5000/predict"
data = {"single_input": [-1, 1, 0, 118, 150000, 0, 1, 38]}
j_data = json.dumps(data)
headers = {"content-type": "application/json", "Accept-Charset": "UTF-8"}
r = requests.post(url, data=j_data, headers=headers)
# print(r)
print(r.json())

# Excercise - Visualization of geospatial data

The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


## imports

In [27]:
import pandas
import geopandas as gpd
import folium
from math import radians, cos, sin, asin, sqrt, pi

## Read data

In [22]:
# Read in the shape file
# Transform to WGS coordinates

# get this file from https://ac.ngi.be/remoteclient-open/SDI/NGI-IGN/fb1e2993-2020-428c-9188-eb5f75e284b9_x-shapefile_31370.zip
# or click through on https://data.gov.be/nl/node/41178
gdf = gpd.read_file("./_data/adminvector72/AD_0_StatisticSector.shp")

# Convert the GeoDataFrame to WGS84 coordinate reference system
gdf = gdf.to_crs({"init": "epsg:4326"})  # change projection to wgs84

## Optimize code

One of the data scientists discovered stackoverflow ;-) and copypasted something from https://gis.stackexchange.com/questions/166820/geopandas-return-lat-and-long-of-a-centroid-point
A data science engineer should be able to speed this next code up

```python
for i in range(0, len(df)):
    df.loc[i,'centroid_lon'] = df.geometry.centroid.x.iloc[i]
    df.loc[i,'centroid_lat'] = df.geometry.centroid.y.iloc[i]
```
Do not run the code as-is it runs for more than 5min.

In [24]:
# explore geo dataframe
gdf.shape
gdf.head(1)

# small test
# df.geometry.centroid.x.iloc[1] returns 4.730839019187626

,tgid,NISCode,ModifDate,Shape_Leng,Shape_Area,geometry
0,{E06A6F81-19B0-4828-A47F-8955CCADA59A},38016X0JQ,2019-10-30,4350.686986,912805.474484,"POLYGON Z ((2.71622 51.15613 0.00000, 2.71707 ..."


In [25]:
%%time
# replace the for loop, 2 new columns should be created
gdf["centroid_lon"] = gdf.geometry.centroid.x
gdf["centroid_lat"] = gdf.geometry.centroid.y
gdf.head(3)

Wall time: 1.95 s


,tgid,NISCode,ModifDate,Shape_Leng,Shape_Area,geometry,centroid_lon,centroid_lat
0,{E06A6F81-19B0-4828-A47F-8955CCADA59A},38016X0JQ,2019-10-30,4350.686986,9.128055e+05,"POLYGON Z ((2.71622 51.15613 0.00000, 2.71707 ...",2.709342,51.150769
1,{44CDC0F6-ED48-43E8-ABC2-69F5B1B7D734},24094B09-,2019-10-30,24003.624570,4.213636e+06,"POLYGON Z ((4.72660 50.98505 0.00000, 4.72696 ...",4.730839,50.977419
2,{7D81EA40-A405-4F74-B4BA-C98FA411CC29},21001A53-,2019-10-30,1215.764322,8.969256e+04,"POLYGON Z ((4.29324 50.83340 0.00000, 4.29337 ...",4.292727,50.831438


## Haversine formula - distance function

Haversine info:
https://www.wikiwand.com/en/Haversine_formula  
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/a65dbbde43ff45bacd2505fcf32b44fc7dcd8cc0)  
φ1, φ2: latitude of point 1 and latitude of point 2 (in radians),  
λ1, λ2: longitude of point 1 and longitude of point 2 (in radians).  
d is the distance between the two points along a great circle of the sphere  
r is the radius of the sphere.  
Because im lazy/efficient I also looked at https://github.com/mapado/haversine/blob/master/haversine/haversine.py

In [28]:
_AVG_EARTH_RADIUS_KM = 6371.0088


def haversine(lat1, lon1, lat2, lon2):
    """ Calculate the great-circle distance (in km) between two points on the Earth surface.
    Takes the latitude and longitude of each point in decimal degrees.
    :param lat1: latitude of first point in decimal degrees
    :param lon1: longitude of first point in decimal degrees
    :param lat2: latitude of second point in decimal degrees
    :param lon2: longitude of second point in decimal degrees
    Example: ``haversine(45.7597, 4.8422, 48.8567, 2.3508)``
    :return: the distance between the two points in km, as a float.
    """
    # get earth radius in required units

    avg_earth_radius = _AVG_EARTH_RADIUS_KM

    lng1 = lon1
    lng2 = lon2

    # convert all latitudes/longitudes from decimal degrees to radians
    lat1, lng1, lat2, lng2 = map(radians, (lat1, lng1, lat2, lng2))

    # calculate d ( the distance between 2 points on a sphere, with the path following the curve of the sphere)
    lat = lat2 - lat1
    lng = lng2 - lng1

    func = sin(lat * 0.5) ** 2 + cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2
    d = 2 * avg_earth_radius * asin(sqrt(func))

    return d

Next, implement some sanity checks for your distance function 

In [29]:
# latitude range 0 - 90
# longitude range 0 - 180

# random input coordinates
lat1 = 75.6
lon1 = 170
lat2 = 10
lon2 = 10

# The distance should always be smaller than half the circumference of the world !
tc = unittest.TestCase("__init__")

half_earth_circumference = 2 * pi * _AVG_EARTH_RADIUS_KM / 2
tc.assertTrue(
    haversine(lat1, lon1, lat2, lon2) < half_earth_circumference,
    f" great-circle distance between 2 points is bigger than half_earth_circumference",
)
tc.assertTrue(
    haversine(lat1, lon1, lat2, lon2) > 0, f" great-circle distance is negative"
)

# half_earth_circumference
# haversine(lat1, lon1, lat2, lon2)

## Stores and nis-districts close to location of interest

In [40]:
# Let's create some variables to indicate the location of your interest
# Data points from GoogleMaps URL
home_lat = 50.873769
home_lon = 4.712944
perimeter_distance = 2  # km

In [41]:
# Figure out which nis districts are near your home
# calculate the distance
gdf["distance"] = gdf[["centroid_lat", "centroid_lon"]].apply(
    lambda x: haversine(x.centroid_lat, x.centroid_lon, home_lat, home_lon), axis=1
)
# Notes
# How to apply a function using 2 columns as input of a pandas df - https://stackoverflow.com/a/52854800/3056345
# Can this be optimized? Currently not required

# nis districts near my home
gdf_nis_centroid_close_by = gdf[gdf["distance"] < perimeter_distance]

In [42]:
# Which 5 stores (from the Data Ingestion excercise) are closest to my home and what is the bird's flight distance?

df_stores = df_clp.rename(
    columns={
        "commercialName": "store",
        "geoCoordinates.latitude": "lat",
        "geoCoordinates.longitude": "lon",
    }
)[
    [
        "store",
        "lat",
        "lon",
        "address.streetName",
        "address.houseNumber",
        "address.postalcode",
        "address.cityName",
        "address.countryName",
        "moreInfoUrl",
    ]
]

df_stores["distance"] = df_stores[["lat", "lon"]].apply(
    lambda x: haversine(x.lat, x.lon, home_lat, home_lon), axis=1
)

df_stores = df_stores.sort_values(by="distance", ascending=True).iloc[:5]

df_stores.head(2)

,store,lat,lon,address.streetName,address.houseNumber,address.postalcode,address.cityName,address.countryName,moreInfoUrl,distance
placeId,,,,,,,,,,
440,KESSEL-LO (COLRUYT),50.870874,4.726720,TIENSESTEENWEG,237,3010,KESSEL-LO,België,https://www.colruyt.be/nl/colruyt-openingsuren...,1.018842
684,LEUVEN (COLRUYT),50.884092,4.701339,LOMBAARDENSTRAAT,2,3000,LEUVEN,België,https://www.colruyt.be/nl/colruyt-openingsuren...,1.407313


## Interactive map

In [43]:
home = [home_lat, home_lon]

# create base map
m = folium.Map(location=home, tiles="OpenStreetMap", zoom_start=14)

# add home marker
folium.Marker(location=home, tooltip="home").add_to(m)

# add perimeter
folium.Circle(
    location=home,
    color="#3186cc",
    fill=False,
    # fill_color='#3186cc',
    tooltip=f"{perimeter_distance}km perimeter from home",
    radius=perimeter_distance * 1000,
).add_to(m)

# add nis centroids
centroids = zip(
    gdf_nis_centroid_close_by.centroid_lat, gdf_nis_centroid_close_by.centroid_lon
)
for centroid in centroids:
    folium.CircleMarker(location=centroid, color="red", radius=1).add_to(m)

# add nis polygon area
folium.Choropleth(
    geo_data=gdf_nis_centroid_close_by.to_json(),
    line_color="grey",
    line_weight=3,
    line_opacity=0.8,
    fill_color="grey",
    fill_opacity=0.2,
    smooth_factor=2.0,  # default: 1.0 More means better performance and smoother look, and less means more accurate representation
    name="nis area",
    overlay=True,
    control=True,
    show=True,
).add_to(m)

folium.LayerControl().add_to(m)

# add stores with popup info
for index, store in df_stores.iterrows():
    popup_html = f"""
                {store.store}                
                <br><br>bird's-eye distance: {round(store.distance,1)}km
                <br><br>{store['address.streetName']}&nbsp;{store['address.houseNumber']}               
                <br>{store['address.postalcode']} {store['address.cityName']}                
                <br><br><a href="{store.moreInfoUrl}">opening hours</a>                
            """
    popup = folium.Popup(popup_html, min_width=1200, close_button=True)
    folium.Marker(
        location=(store.lat, store.lon),
        tooltip=f"{store.store}",
        popup=popup,
        icon=folium.Icon(color="orange", prefix="fa", icon="shopping-cart"),
    ).add_to(m)

# show map
m